In [1]:
# ! pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup as bs
from datetime import datetime
import pandas as pd
import time


In [ ]:
# Chrome 옵션 설정
options = Options()
options.binary_location = "./chrome-win64/chrome.exe"  # chrome 경로 설정
options.add_argument("--no--sandbox") # sandbox 설정
options.add_argument("--disable-dev-shm-usage")  # 공유메모리 설정
options.add_argument("--window-size=1920,1080") # (화면크기)해상도 설정, 반응형 웹 대비
# options.add_argument("--disable-gpu") # GPU 비활성화 

# ChromeDriver 설정
service = Service("chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.starbucks.co.kr/index.do")
print(driver.title)

# 요소찾기
hover_element = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a")
menu_option = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a")

# 마우스 호버 액션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform() # 마우스 올린거 처럼

# 약간의 시간을 기다린 후 클릭
time.sleep(1)
menu_option.click()
time.sleep(5)

# 입력창 선택 후 검색
driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a").click() # '지역검색' 을 선택
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, "#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(6) > a").click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, "#mCSB_2_container > ul > li:nth-child(1) > a").click()
time.sleep(1)

html = driver.page_source
# print(html)
driver.quit() # 브라우저 종료

In [11]:
# 현재 페이지 정보 가져오기
star_list=[]
soup = bs(html, 'html.parser')
# 본문내용 가져오기
contents = soup.select("#mCSB_3_container > ul > li")

test = contents[0]
print(test.select_one('p.result_details').text, test.select_one('i.pin_generalDT').text)
print(test['data-name'],'/' ,test['data-lat'], '/' ,test['data-long'])



for c in contents:
    result = (c['data-name'], c['data-lat'], c['data-long'], c.select_one('p.result_details').text, c.select_one('i')['class'][0])
    star_list.append(result)
# print(star_list)

print("=========================")
print(star_list[0][3])



부산광역시 금정구 중앙대로 1989 (남산동)1522-3232 리저브 매장 2번
금정남산동DT / 35.2621786292157 / 129.091325043351
부산광역시 금정구 중앙대로 1989 (남산동)1522-3232


In [18]:
columns = ['이름','위도', '경도','주소','타입']
sb = pd.DataFrame(star_list, columns=columns)
sb.head()

,이름,위도,경도,주소,타입
0,금정남산동DT,35.2621786292157,129.091325043351,부산광역시 금정구 중앙대로 1989 (남산동)1522-3232,pin_generalDT
1,금정구청DT,35.2388135211636,129.092985849499,부산광역시 금정구 중앙대로 1730 (부곡동)1522-3232,pin_generalDT
2,온천장역DT,35.224006,129.08656,부산광역시 금정구 식물원로 13 (장전동)1522-3232,pin_generalDT
3,부산대정문,35.23212165114605,129.0844959292725,부산광역시 금정구 부산대학로64번길 12 (장전동)1522-3232,pin_general
4,부산대역,35.23052180284867,129.08869433543111,"부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)1522-3232",pin_general


In [13]:
sb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   name    144 non-null    object
 1   위도      144 non-null    object
 2   경도      144 non-null    object
 3   주소      144 non-null    object
 4   타입      144 non-null    object
dtypes: object(5)
memory usage: 5.8+ KB


In [19]:
timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
csv_path = f'starbuck_{timestamp}.csv'
sb.to_csv(csv_path, index=True)
print("csv 파일 생성완료")

csv 파일 생성완료


In [ ]:
# ! pip install sqlalchemy

In [15]:
# mysql 에서 데이터 가져와 dataframe 로 변환하기
from sqlalchemy import create_engine

user = "root"
password = "1234"
host = "210.119.14.60"
port = "3306"
database = "kgy7820"

total = f"{user}:{password}@{host}:{port}/{database}"
print(total)
engine = create_engine('mysql+mysqlconnector://'+total)

query = "SELECT * FROM kgy"
df = pd.read_sql(query,engine)
df.head()

root:1234@210.119.14.60:3306/kgy7820


,id,name,detail,type,lat,lng
0,1,금정남산동DT,부산광역시 금정구 중앙대로 1989 (남산동)1522-3232,pin_generalDT,35.262179,129.091325
1,2,금정구청DT,부산광역시 금정구 중앙대로 1730 (부곡동)1522-3232,pin_generalDT,35.238814,129.092986
2,3,온천장역DT,부산광역시 금정구 식물원로 13 (장전동)1522-3232,pin_generalDT,35.224006,129.086560
3,4,부산대정문,부산광역시 금정구 부산대학로64번길 12 (장전동)1522-3232,pin_general,35.232122,129.084496
4,5,부산대역,"부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)1522-3232",pin_general,35.230522,129.088694


In [16]:
df.columns = ['name','lat','lng','detail', 'type','id']
df.head()

,name,lat,lng,detail,type,id
0,1,금정남산동DT,부산광역시 금정구 중앙대로 1989 (남산동)1522-3232,pin_generalDT,35.262179,129.091325
1,2,금정구청DT,부산광역시 금정구 중앙대로 1730 (부곡동)1522-3232,pin_generalDT,35.238814,129.092986
2,3,온천장역DT,부산광역시 금정구 식물원로 13 (장전동)1522-3232,pin_generalDT,35.224006,129.086560
3,4,부산대정문,부산광역시 금정구 부산대학로64번길 12 (장전동)1522-3232,pin_general,35.232122,129.084496
4,5,부산대역,"부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)1522-3232",pin_general,35.230522,129.088694
